アイコンを取得

In [9]:
%%bash

curl -LO https://desktop.icons8.com/updates/mac/Icons8App_for_Mac_OS.dmg
7z x Icons8App_for_Mac_OS.dmg
7z x 2.hfs
tar zxvf Icons8*/Icons8.app/Contents/Resources/icons.tar.gz
ls icons


7-Zip [64] 9.20  Copyright (c) 1999-2010 Igor Pavlov  2010-11-18
p7zip Version 9.20 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,4 CPUs)

Processing archive: Icons8App_for_Mac_OS.dmg

Extracting  0.ddm
Extracting  1.Apple_partition_map
Extracting  2.hfs

Everything is Ok

Files: 3
Size:       118489088
Compressed: 83487690

7-Zip [64] 9.20  Copyright (c) 1999-2010 Igor Pavlov  2010-11-18
p7zip Version 9.20 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,4 CPUs)

Processing archive: 2.hfs

Extracting  Icons8 v5.6.5
Extracting  Icons8 v5.6.5/.background
Extracting  Icons8 v5.6.5/.DS_Store
Extracting  Icons8 v5.6.5/.HFS+ Private Directory Data
Extracting  Icons8 v5.6.5/Applications
Extracting  Icons8 v5.6.5/Icons8.app
Extracting  Icons8 v5.6.5/[]
Extracting  Icons8 v5.6.5/.background/backgroundImage.tiff
Extracting  Icons8 v5.6.5/Icons8.app/Contents
Extracting  Icons8 v5.6.5/Icons8.app/Contents/_CodeSignature
Extracting  Icons8 v5.6.5/Icons8.app/Contents/Frameworks
Extracting  Icons8 v5.6.5/Icons

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 79.6M  100 79.6M    0     0   148M      0 --:--:-- --:--:-- --:--:--  148M


In [1]:
import plyvel
from svglib.svglib import svg2rlg
from reportlab.graphics import renderPM
import re
from io import BytesIO
import shutil
from PIL import Image
import os
import json
import unicodedata

In [10]:
# Adjust to your local path:
path = 'icons'
db = plyvel.DB(path)

In [11]:
splitter = re.compile(b'[\x00-\x08]')

def parse_value(value):
    res = {}
    prev = ''
    for elem in splitter.split(value):
        if not elem:
            continue
        try:
            elem = elem.decode('utf8')
        except UnicodeDecodeError:
            continue
        if elem in ('category', 'name', 'platform', 'canonical_name', 'svg'):
            if elem == 'name' and len(prev) == 1:
                prev = 'u_' + unicodedata.name(prev).lower().replace(' ', '_')
            res[elem] = prev
        prev = elem
    return res

for _, value in db:
    res = parse_value(value)
    break
res

{'canonical_name': 'Address Book',
 'category': 'Messaging',
 'name': 'address_book',
 'platform': 'windows',
 'svg': '<?xml version="1.0" encoding="utf-8"?>\n<!-- Generator: Adobe Illustrator 15.0.0, SVG Export Plug-In . SVG Version: 6.00 Build 0)  -->\n<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">\n<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" id="Layer_1" x="0px" y="0px" width="26px" height="26px" viewBox="0 0 26 26" enable-background="new 0 0 26 26" xml:space="preserve">\n\n<path fill="#000000" d="M12.792,9.803c-0.221,0-0.423,0.053-0.618,0.166c-0.2,0.111-0.399,0.302-0.594,0.562  c-0.198,0.266-0.359,0.593-0.48,0.977c-0.121,0.385-0.184,0.741-0.184,1.061c0,0.488,0.115,0.867,0.34,1.126  c0.371,0.428,0.811,0.485,1.343,0.205c0.225-0.121,0.444-0.304,0.653-0.543c0.207-0.241,0.382-0.553,0.517-0.929  c0.136-0.378,0.206-0.759,0.206-1.137c0-0.481-0.119-0.855-0.354-1.112C13.393,9.926,13.12,9.

In [12]:
icons = {}

for key, value in db:
    try:
        res = parse_value(value)
    except ValueError:
        continue
    if res.get('platform') == 'ios':
        name = res.get('name')
        if not name:
            name = res.get('canonical_name')
            if not name:
                continue
            name = name.lower().replace(' ', '_')
        icons[name] = res
len(icons)

12420

In [13]:
SIZES = (16, 28, 32, 50)

if os.path.isdir('icons'):
    shutil.rmtree('icons')
os.mkdir('icons')
for size in SIZES:
    os.mkdir('icons/png%s' % size)
os.mkdir('icons/svg')

In [50]:
from svglib.svglib import SvgRenderer
from xml.dom import minidom
from xml.parsers import expat

def svg2rlg(xml_str):
    try:
        svg = minidom.parseString(xml_str)
    except expat.ExpatError:
        raise ValueError
    svgRenderer = SvgRenderer()
    svgRenderer.render(svg)
    drawing = svgRenderer.finish()
    return drawing

In [66]:
from svglib.svglib import svg2rlg

In [71]:
saved = []
for icon in icons.values():
    icon = dict(icon)
    if not 'svg' in icon:
        continue
    svg = icon.pop('svg')
    try:
        drawing = svg2rlg(BytesIO(svg.encode('utf8')))
    except ValueError:
        continue
    except AttributeError:
        continue
    except TypeError:
        print("TypeError: %s" % icon['name'])
    open('icons/svg/%s.svg' % icon['name'], 'w').write(svg)
    p = renderPM.drawToPIL(drawing)
    for size in SIZES:
        resized = p.resize((size, size), Image.ANTIALIAS)
        resized.save('icons/png%s/%s.png' % (size, icon['name']))
    saved.append(icon)
json.dump(saved, open('icons/index.json', 'w'), indent=2)
len(saved)
    

TypeError: mario_8_bit
TypeError: highway
TypeError: temple
TypeError: manga
TypeError: douban
TypeError: battelefied_filled
TypeError: data_matrix_code_
TypeError: qt
TypeError: nintendo_with_card
TypeError: hookah
TypeError: mario_8_bit_filled
TypeError: teenager_male
TypeError: sewing_scissors
TypeError: owl
TypeError: animation_rig_filled
TypeError: werewolf
TypeError: open_connect
TypeError: es_file_explorer
TypeError: werewolf_filled
TypeError: three_wheel_car
TypeError: data_matrix_code_filled
TypeError: lettering
TypeError: reliance_digital_tv
TypeError: sport_bottle_filled
TypeError: 2_position_switch
TypeError: safari_hat


Can't handle color: url(#SVGID_1_)


12401